In [1]:
import sys

# runtype = "daily"
# report_date = "2020-04-30" #automated

no_of_cpu = 4
max_cores = 4
executor_mem = '26g'

Job_Name = "Mayur :COE_Endorsement_Prod"

print ("JOB NAME = "+str(Job_Name))
print ("No_of_cpu="+str(no_of_cpu))
print ("Max_cores="+str(max_cores))
print ("Executor_mem="+str(executor_mem))
# print ("Runtype="+str(runtype))
# print ("Report_date="+str(report_date))


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'gpanalytics'
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'
spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
conf.set('spark.executor.memory',executor_mem)
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','120m')
conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('spark.executor.heartbeatInterval','120s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70");
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer");
conf.set("spark.broadcast.compress", "true");
conf.set("spark.shuffle.compress", "true");
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.driver.maxResultSize", "4g");
conf.set("spark.app.name", Job_Name)
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','12g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','300')
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
conf.set("spark.scroll.keepalive","120m")

# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark import SparkContext, SparkConf
conf = pyspark.SparkConf()
#import numpy
import calendar
import time
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, StringType, DecimalType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
import re # to use regular expressions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
557,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JOB NAME = Mayur :COE_Endorsement_Prod
No_of_cpu=4
Max_cores=4
Executor_mem=26g

In [2]:
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import time
from time import gmtime, strftime
import psycopg2
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2022-12-21 10:12:43.583589'

In [3]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

# Define the GPDB-Python connection options for PROD 

conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_gpdb_jdbc(col_str,dbtable,dbschema='public',col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='customer_activity')\
            .filter(col('source')=='activity2')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2021, 5, 14, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                
#         df_progress_values = [{"table_name":table_name,"source":source,"from_datetime":time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S")
#                         ,"to_datetime":time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),'records':records,
#                          'start_time':start_time}]

        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        
#         schema = StructType([StructField("table_name", StringType(),True),StructField("source", StringType(),True),\
#                      StructField("from_datetime", StringType(),True),StructField("to_datetime", StringType(),True),\
#                      StructField("records", IntegerType(),True),StructField("start_time", StringType(),True)])

#         df_progress = sqlContext.createDataFrame(df_progress_values,schema)

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark
def update_activity(conn_to,table_name,source,gpdb_spark_options = prod_gpdb_spark_options,
                  schema="customermart"):
    conn_to.rollback()
    primary_key = ['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp']
    update_fields = ['feature_string_1', 'feature_string_2', 'feature_string_3', 'feature_string_4',
               'feature_date_1', 'feature_date_2']
    df_1 = sqlContext.read.format("greenplum").options(**gpdb_spark_options).option("dbschema",schema).option("dbtable","{}_2_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.{table_name}_2_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_2_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_2_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import functools 
# Function row-wise joining of dataframes
def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2  #T currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2022, 11, 30, 0, 0), 'end_date': datetime.datetime(2022, 12, 19, 0, 0)}

In [10]:
listValues = ['DECLARATION','NEWPOLICY','PROPOSALMODIFICATION']

col_str=['txt_customer_id','txt_policy_no_char','num_reference_number','txt_endorsement_wording',
'dat_endorsement_eff_date','tgt_commit_timestamp','txt_modeofoperation']


gc_info_df=load_gpdb_jdbc(col_str,"policy_gc_gen_prop_information_tab",'public','tgt_commit_timestamp',time_filter)
gc_info_df=gc_info_df.filter((col('txt_policy_no_char').isNotNull())).filter((col('txt_customer_id').isNotNull()))\
                    .filter((col('txt_modeofoperation').isNotNull())).filter(~gc_info_df.txt_modeofoperation.isin(listValues))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
col_str=['num_reference_number','num_endorsement_type_code']

gc_dist_df=load_gpdb_jdbc(col_str,"underwriting_gc_distribution_channel_tab")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
col_str=['trans_id','status']

gc_trans_df=load_gpdb_jdbc(col_str,"underwriting_gc_cnfgtr_d_all_transactions")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_1= gc_info_df.join(gc_dist_df, gc_info_df.num_reference_number == gc_dist_df.num_reference_number, "left_outer")\
.drop( gc_dist_df.num_reference_number)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_2= df_1.join(gc_trans_df, df_1.num_reference_number == gc_trans_df.trans_id, "left_outer")\
.drop(gc_trans_df.trans_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_2.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['txt_customer_id', 'txt_policy_no_char', 'num_reference_number', 'txt_endorsement_wording', 'dat_endorsement_eff_date', 'tgt_commit_timestamp', 'txt_modeofoperation', 'num_endorsement_type_code', 'status']

In [16]:

df_2=df_2.withColumn('activity_type', lit('Endorsement'))\
.withColumn('source_system_name', lit('GC'))\
.withColumn('source_system_customer_id',col('txt_customer_id'))\
.withColumn('reference_number',col('num_reference_number'))\
.withColumn('feature_string_3',col('status'))\
.withColumn('feature_string_4',col('txt_endorsement_wording'))\
.withColumn('feature_date_1',col('dat_endorsement_eff_date'))\
.withColumn('feature_date_2',col('dat_endorsement_eff_date'))\
.withColumn('commit_timestamp',col('tgt_commit_timestamp'))



# 'GC' as source_system_name,
# info.txt_customer_id as source_system_customer_id,
# 'Endorsement' as activity_type,
# info.txt_policy_no_char as activity_id,
# info.num_reference_number as reference_number,
# dist.num_endorsement_type_code, -- need for joining
# transac.status as feature_string_3,
# info.txt_endorsement_wording as feature_string_4,
# info.dat_endorsement_eff_date as feature_date_1,
# info.dat_endorsement_eff_date as feature_date_2,
# info.tgt_commit_timestamp as commit_timestamp

# activity5=activity5.withColumn('commit_timestamp', col('tgt_commit_timestamp'))
# activity5=activity5.withColumn('feature_string_1', col('txt_cancellation_reason_comm'))
# activity5=activity5.withColumn('feature_date_1', col('dat_endorsement_eff_date'))
# activity5=activity5.withColumn('feature_value_2', col('num_total_premium').cast('float'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
col_str=['endt_type_status','txt_endt_type_desc','num_endt_type_cd']

endorse_df=load_gpdb_jdbc(col_str,"reference_gc_uw_endorsement_type_master")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
endorse_df=endorse_df.withColumn("feature_string_1",expr("case when endt_type_status = 'NILENDORSEMENT' then 'Non Financial'\
when endt_type_status = 'NOTNILENDORSEMENT' then 'Financial'\
else null end as feature_string_1")).alias("feature_string_1")\
.withColumn('feature_string_2',col('txt_endt_type_desc'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
final_df = df_2.join(endorse_df,df_2.num_endorsement_type_code==endorse_df.num_endt_type_cd,"left")
# final_df = final_df.drop_duplicates(['source_system_customer_id','activity_type', 'activity_id'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
final_df=final_df.withColumn('activity_id',concat_ws('|',trim(col('txt_policy_no_char')),hash(trim(col('feature_string_1')),trim(col('feature_string_2')))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# w = Window.partitionBy('source_system_customer_id','activity_id').orderBy(F.col('commit_timestamp').desc())
# final_df = final_df.withColumn('row',F.row_number().over(w))
# final_df = final_df.filter(F.col('row')==1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w = Window.partitionBy('source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id').orderBy(col('commit_timestamp').desc())
final_df = final_df.withColumn('row',F.row_number().over(w))
final_df = final_df.filter(F.col('row')==1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
final_df=final_df.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_type').cast('string'), col('activity_id').cast('string'),\
                         col('feature_string_1').cast('string'),col('feature_string_2').cast('string'),col('feature_string_3').cast('string'),\
                         col('feature_string_4').cast('string'),col('feature_date_1').cast(DateType()),col('feature_date_2').cast('timestamp'),col('commit_timestamp').cast("timestamp"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
final_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=final_df.count()
SrcGCIDCount=str(final_df.select(countDistinct("source_system_customer_id")).collect()[0][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2410456

In [26]:
output_index = "customer_activity_2_staging"
schema = "customermart"

import sys
try:
    
    final_df.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [27]:
from pg import DB
db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
GCidCount = db.query("select count(distinct source_system_customer_id) from customermart.customer_activity_2_staging").getresult()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
table_name='customer_activity'
source='activity2'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
update_activity(conn_to,table_name,source,prod_gpdb_spark_options,'customermart')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Inside try segment
Executed Only insert query

In [30]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.customer_activity",
         "server.port":"1150-1170"}


activity2_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('source_system_name','source_system_customer_id','activity_type', 'activity_id', 'feature_string_1', 
               'feature_string_2', 'feature_string_3', 'feature_string_4','feature_date_1', 'feature_date_2', 'commit_timestamp')\
.filter(col('activity_type')=='Endorsement')\
.filter(col('commit_timestamp').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
SrcGCIDCount=str(final_df.select(countDistinct("source_system_customer_id")).collect()[0][0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_MissingData=final_df.exceptAll(activity2_df).withColumn('DataMisMatch',lit('Missing values at Target'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
reconresult=[{
    "SrcCount":SrcGCIDCount,
    "TargetCount":GCidCount[0][0],
    "TrgtMissMatchCount":df_MissingData.count()
}]

from pyspark.sql import Row
df_recon=sqlContext.createDataFrame(Row(**x) for x in reconresult)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_recon.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+------------------+
|SrcCount|TargetCount|TrgtMissMatchCount|
+--------+-----------+------------------+
| 1043674|    1043674|                 0|
+--------+-----------+------------------+

In [35]:
df_recon=df_recon.withColumn('Status',when((df_recon.SrcCount==df_recon.TargetCount) & (df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))
status=df_recon.select(df_recon.Status).collect()[0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
update_progress(table_name,source,time_filter,records,start_time,starttime,status)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Updated Progress for customer_activity from activity2 until 2022-12-19 00:00:00. 2410456 records processed in this run
success

In [37]:
final_df.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, activity_type: string, activity_id: string, feature_string_1: string, feature_string_2: string, feature_string_3: string, feature_string_4: string, feature_date_1: date, feature_date_2: timestamp, commit_timestamp: timestamp]

In [38]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…